In [13]:
# importing libraries

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

import torch 
from torch import nn 

In [14]:
# Define the batch size and model save paths

BATCH_SIZE = 128
PATH = 'Models/02_Sequential_10M/'

In [15]:
# Loading the dataset

data = pd.read_csv('end_coordinates.csv')
data.head()

,x,y,z
0,0.053,0.196,0.1065
1,0.053,0.195,0.1065
2,0.053,0.194,0.1065
3,0.053,0.193,0.1065
4,0.052,0.193,0.1065


In [16]:
# Normalising the data 

scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data))
data.columns = ['x','y','z']
data.head()

,x,y,z
0,0.149425,1.000000,0.0
1,0.149425,0.994872,0.0
2,0.149425,0.989744,0.0
3,0.149425,0.984615,0.0
4,0.146552,0.984615,0.0


In [17]:
# Device agnostic code

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
device

'cuda'

In [18]:
# Converting data to tensors 

coordinates = torch.Tensor(data[['x','y','z']].to_numpy())

coordinates.shape

torch.Size([3891, 3])

In [19]:

# Defining the model 

class InverseKinematicsABBIRB140(nn.Module):
    
    """ This is a model structure specifically designed to calculate the inverse kinematics of the industrial robot ABB IRB 140, but can be used for any robot
        This has a layer structure of input -> 64 -> 128 -> 64 -> output with non-linear activation and normalisation layers
    Args:
        input_features (int): defines the number of features given as input to the model
        output_features (int): defines the number of features the model outputs
    """
    
    def __init__(self, input_features, output_features):
        super().__init__()
        self.layer_stack_1 = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout()
        )
        self.layer_stack_2 = nn.Sequential(
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout()
        )
        self.layer_stack_3 = nn.Sequential(
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout()
        )
        self.layer_stack_4 = nn.Sequential(
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout(),
            nn.Linear(in_features=128, out_features=output_features)
        )
    
    def forward(self, x):
        x = self.layer_stack_1(x)
        x = self.layer_stack_2(x)
        x = self.layer_stack_3(x)
        x = self.layer_stack_4(x)
        return x   
    

In [20]:
# Instantiating various models used 

model_0 = InverseKinematicsABBIRB140(input_features=3,output_features=1)
model_1 = InverseKinematicsABBIRB140(input_features=4,output_features=1)
model_2 = InverseKinematicsABBIRB140(input_features=5,output_features=1)
model_3 = InverseKinematicsABBIRB140(input_features=6,output_features=1)
model_4 = InverseKinematicsABBIRB140(input_features=7,output_features=1)
model_5 = InverseKinematicsABBIRB140(input_features=8,output_features=1)

model_0.load_state_dict(torch.load(f=PATH+"model_0.pth"))
model_1.load_state_dict(torch.load(f=PATH+"model_1.pth"))
model_2.load_state_dict(torch.load(f=PATH+"model_2.pth"))
model_3.load_state_dict(torch.load(f=PATH+"model_3.pth"))
model_4.load_state_dict(torch.load(f=PATH+"model_4.pth"))
model_5.load_state_dict(torch.load(f=PATH+"model_5.pth"))

<All keys matched successfully>

In [21]:
# Calculate all the joint angles from our models

def get_joint_angles(end_effector_coordinates:torch.Tensor) -> torch.Tensor:
    model_0.eval()
    with torch.inference_mode():
        joint_0_pred = model_0(end_effector_coordinates)
        
    model_1.eval()
    with torch.inference_mode():
        joint_1_pred = model_1(torch.cat((end_effector_coordinates, joint_0_pred), dim=1))
        
    model_2.eval()
    with torch.inference_mode():
        joint_2_pred = model_2(torch.cat((end_effector_coordinates, joint_0_pred, joint_1_pred), dim=1))
        
    model_3.eval()
    with torch.inference_mode():
        joint_3_pred = model_3(torch.cat((end_effector_coordinates, joint_0_pred, joint_1_pred, joint_2_pred), dim=1))
        
    model_4.eval()
    with torch.inference_mode():
        joint_4_pred = model_4(torch.cat((end_effector_coordinates, joint_0_pred, joint_1_pred, joint_2_pred, joint_3_pred), dim=1))
        
    model_5.eval()
    with torch.inference_mode():
        joint_5_pred = model_5(torch.cat((end_effector_coordinates, joint_0_pred, joint_1_pred, joint_2_pred, joint_3_pred, joint_4_pred), dim=1))
        
    return torch.cat((joint_0_pred, joint_1_pred, joint_2_pred, joint_3_pred, joint_4_pred, joint_5_pred), dim=1)

In [23]:
ypred = get_joint_angles(coordinates)
ypred

tensor([[0.8429, 0.7719, 0.5464, 0.5046, 0.5083, 0.5126],
        [0.8434, 0.7725, 0.5463, 0.5046, 0.5078, 0.5126],
        [0.8438, 0.7731, 0.5462, 0.5045, 0.5073, 0.5126],
        ...,
        [0.3270, 0.7681, 0.5585, 0.5617, 0.4039, 0.5118],
        [0.3280, 0.7691, 0.5581, 0.5603, 0.4055, 0.5118],
        [0.3312, 0.7722, 0.5569, 0.5572, 0.4127, 0.5117]])

In [24]:
output = pd.DataFrame(ypred.numpy(), columns=['joint_0','joint_1','joint_2','joint_3','joint_4','joint_5'])
output.to_csv('joint_angles.csv', index=False)